In [8]:
import pymysql
pymysql.install_as_MySQLdb()

# Connect
myhost='localhost'
myuser='myuser'
mypasswd='mypass'
mydb='mydb'
connection = pymysql.connect(host=myhost,
                             user=myuser,
                             passwd=mypasswd,
                             db=mydb)

#print ("connect successful!!")

text = "A hacker gained access to the PHI for the covered entity's patients."
 
try:
    with connection.cursor() as cursor:
        # Execute SQL select statement
        cursor.execute("SELECT instance, class FROM simple_types where instance like '%Raleigh%' and class like '%CITY%'")
        # Commit your changes if writing
        # In this case, we are only reading data
        # db.commit()
        
        # Get the number of rows in the resultset
        numrows = cursor.rowcount
        
        # Get and display one row at a time
        for x in range(0, numrows):
            row = cursor.fetchone()
            print(row[0], "-->", row[1])
            
# Close the connection
finally:
    # Close connection.
    connection.close()

<Raleigh_City_Council> --> <wikicat_City_councils_in_the_United_States>
<Raleigh_Aitchison> --> <wikicat_Oklahoma_City_Indians_players>
<Sim_Raleigh> --> <wikicat_Hull_City_A.F.C._players>
<Walter_Raleigh_(professor)> --> <wikicat_People_educated_at_the_City_of_London_School>
<Raleigh_DeGeer_Amyx> --> <wikicat_People_from_Kansas_City,_Kansas>
<Raleigh_Croshaw> --> <wikicat_People_from_Elizabeth_City_County,_Virginia>


In [12]:
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('C:/Users/Nirav/libraries/stanford-corenlp-full')
connection = pymysql.connect(host=myhost,
                             user=myuser,
                             passwd=mypasswd,
                             db=mydb)

try: 
    with connection.cursor() as cursor:
        #cursor.execute("Select text, day, place, attacker, target, instrument from cyberattacks")
        cursor.execute("Select 'ISIS members threw stones in Paris.'")
        numrows = cursor.rowcount
        for x in range(0, numrows):
            row = cursor.fetchone()
            sentence = row[0]

            print('Sentence:', sentence)
            print('Tokenize:', nlp.word_tokenize(sentence))
            print('Part of Speech:', nlp.pos_tag(sentence))
            print('Named Entities:', nlp.ner(sentence))
            print('Constituency Parsing:', nlp.parse(sentence))
            print('Dependency Parsing:', nlp.dependency_parse(sentence))
finally: 
    connection.close()
    nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Sentence: ISIS members threw stones in Paris.
Tokenize: ['ISIS', 'members', 'threw', 'stones', 'in', 'Paris', '.']
Part of Speech: [('ISIS', 'NNP'), ('members', 'NNS'), ('threw', 'VBD'), ('stones', 'NNS'), ('in', 'IN'), ('Paris', 'NNP'), ('.', '.')]
Named Entities: [('ISIS', 'ORGANIZATION'), ('members', 'O'), ('threw', 'O'), ('stones', 'O'), ('in', 'O'), ('Paris', 'CITY'), ('.', 'O')]
Constituency Parsing: (ROOT
  (S
    (NP (NNP ISIS) (NNS members))
    (VP (VBD threw)
      (NP (NNS stones))
      (PP (IN in)
        (NP (NNP Paris))))
    (. .)))
Dependency Parsing: [('ROOT', 0, 3), ('compound', 2, 1), ('nsubj', 3, 2), ('dobj', 3, 4), ('case', 6, 5), ('nmod', 3, 6), ('punct', 3, 7)]


In [21]:
# importing StandfordCoreNLP to tokenize, tag, and ner
nlp = StanfordCoreNLP('C:/Users/Nirav/libraries/stanford-corenlp-full')

sentence = "ISIS members threw stones in Paris."
sentence_tokens = nlp.word_tokenize(sentence)
sentence_tags = nlp.pos_tag(sentence)
sentence_ner = nlp.ner(sentence)
#sentence_parse = nlp.parse(sentence)
#sentence_dependency = nlp.dependency_parse(sentence)

to_replace_ners = []
to_replace_verbs = []

for (i, j) in sentence_ner:
    #print(i, j)
    if(j!='O'):
        print(i, j)
        to_replace_ners.append((i,j))
        
verb_check = 0
        
for (i, j) in sentence_tags:
    if(verb_check == 1):
        verb = verb + '_' + i
        to_replace_verbs = [verb]
        verb_check = 0
        print(verb)
    
    if(j=='VBD'):
        print(i, j)
        verb_check = 1
        verb = i
    
        
nlp.close()

ISIS ORGANIZATION
Paris CITY
threw VBD


In [ ]:
# Importing word2vec to find similarity and neighboring words
import gensim
from gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Nirav/libraries/GoogleNews-vectors-negative300.bin.gz', binary=True) 

topk = 10
replacement_ners = []
replacement_verbs = []

for (i, j) in to_replace_ners:
    similar_ners = model.most_similar(i, topk)
    replacements_ners.append((i, similar_ners))

print(replacement_ners)
    
for verb in replacements_verbs:
    similar_verbs = model.most_similar(verb, topk)
    replacement_verbs.append((verb,similar_verbs))

print(replacement_verbs)